In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [6]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]
print(links)

[<a href="/en/squads/b8fd03ef/Manchester-City-Stats">Manchester City</a>, <a href="/en/matches/45bb8cac/Brighton-and-Hove-Albion-Manchester-City-April-25-2024-Premier-League" style="color:#fff; text-decoration:none; background-color: transparent">W</a>, <a href="/en/matches/80bbb25e/Nottingham-Forest-Manchester-City-April-28-2024-Premier-League" style="color:#fff; text-decoration:none; background-color: transparent">W</a>, <a href="/en/matches/5a9032bf/Manchester-City-Wolverhampton-Wanderers-May-4-2024-Premier-League" style="color:#fff; text-decoration:none; background-color: transparent">W</a>, <a href="/en/matches/4e211d29/Fulham-Manchester-City-May-11-2024-Premier-League" style="color:#fff; text-decoration:none; background-color: transparent">W</a>, <a href="/en/matches/2ba17e6d/Tottenham-Hotspur-Manchester-City-May-14-2024-Premier-League" style="color:#fff; text-decoration:none; background-color: transparent">W</a>, <a href="/en/players/1f44ac21/Erling-Haaland">Erling Haaland</a>, 

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
data = requests.get(team_urls[0])

In [ ]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7780\4209044294.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


In [ ]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
shooting = pd.read_html(data.text, match="Shooting")[0]

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7780\2517714723.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [ ]:
shooting.head()

For Arsenal                                                              \
         Date   Time              Comp                Round  Day    Venue   
0  2023-08-06  16:00  Community Shield  FA Community Shield  Sun  Neutral   
1  2023-08-12  12:30    Premier League          Matchweek 1  Sat     Home   
2  2023-08-21  20:00    Premier League          Matchweek 2  Mon     Away   
3  2023-08-26  15:00    Premier League          Matchweek 3  Sat     Home   
4  2023-09-03  16:30    Premier League          Matchweek 4  Sun     Home   

                                           ... Standard                \
  Result     GF     GA           Opponent  ...     Dist   FK PK PKatt   
0      D  1 (4)  1 (1)    Manchester City  ...      NaN  NaN  0     0   
1      W      2      1  Nottingham Forest  ...     19.1  0.0  0     0   
2      W      1      0     Crystal Palace  ...     16.4  0.0  1     1   
3      D      2      2             Fulham  ...     13.8  0.0  1     1   
4      W      3      1  Manchester United  ...     15.0  0.0  0     0   

  Expected                           Unnamed: 25_level_0  
        xG npxG npxG/Sh G-xG np:G-xG        Match Report  
0      NaN  NaN     NaN  NaN     NaN        Match Report  
1      0.8  0.8    0.06  1.2     1.2        Match Report  
2      2.0  1.2    0.09 -1.0    -1.2        Match Report  
3      3.2  2.4    0.14 -1.2    -1.4        Match Report  
4      2.3  2.3    0.13  0.7     0.7        Match Report  

[5 rows x 26 columns]

In [ ]:
shooting.columns = shooting.columns.droplevel()

In [ ]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [ ]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (4),1 (1),Manchester City,...,4-3-3,Stuart Attwell,Match Report,Arsenal won on penalty kicks following normal ...,7,3,NaN,NaN,0,0
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2,1,Nottingham Forest,...,4-3-3,Michael Oliver,Match Report,NaN,15,7,19.1,0.0,0,0
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1,0,Crystal Palace,...,4-3-3,David Coote,Match Report,NaN,13,2,16.4,0.0,1,1
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2,2,Fulham,...,4-3-3,Paul Tierney,Match Report,NaN,18,9,13.8,0.0,1,1
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3,1,Manchester United,...,4-3-3,Anthony Taylor,Match Report,NaN,17,5,15.0,0.0,0,0


In [ ]:
years = list(range(2022, 2020, -1))
all_matches = []

In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
import time
try:
    for year in years:
        data = requests.get(standings_url)
        soup = BeautifulSoup(data.text)
        standings_table = soup.select('table.stats_table')[0]

        links = [l.get("href") for l in standings_table.find_all('a')]
        links = [l for l in links if '/squads/' in l]
        team_urls = [f"https://fbref.com{l}" for l in links]
        
        previous_season = soup.select("a.prev")[0].get("href")
        standings_url = f"https://fbref.com{previous_season}"
        
        for team_url in team_urls:
            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
            data = requests.get(team_url)
            matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
            soup = BeautifulSoup(data.text)
            links = [l.get("href") for l in soup.find_all('a')]
            links = [l for l in links if l and 'all_comps/shooting/' in l]
            data = requests.get(f"https://fbref.com{links[0]}")
            shooting = pd.read_html(data.text, match="Shooting")[0]
            shooting.columns = shooting.columns.droplevel()
            try:
                team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
            except ValueError:
                continue
            team_data = team_data[team_data["Comp"] == "Premier League"]
            
            team_data["Season"] = year
            team_data["Team"] = team_name
            all_matches.append(team_data)
            time.sleep(1)
except:
    pass

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7780\1338002569.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7780\1338002569.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7780\1338002569.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_7780\1338002569.py:22: FutureWarning: Passing literal html

ImportError: Missing optional dependency 'html5lib'.  Use pip or conda to install html5lib.

In [ ]:
len(all_matches)

14

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2,1,Nottingham Forest,...,Match Report,NaN,15.0,7.0,19.1,0.0,0,0,2022,Arsenal
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1,0,Crystal Palace,...,Match Report,NaN,13.0,2.0,16.4,0.0,1,1,2022,Arsenal
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2,2,Fulham,...,Match Report,NaN,18.0,9.0,13.8,0.0,1,1,2022,Arsenal
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3,1,Manchester United,...,Match Report,NaN,17.0,5.0,15.0,0.0,0,0,2022,Arsenal
5,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1,0,Everton,...,Match Report,NaN,13.0,4.0,17.4,0.0,0,0,2022,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,2024-04-24,20:00,Premier League,Matchweek 29,Wed,Home,W,4,2,Sheffield United,...,Match Report,NaN,24.0,12.0,16.9,0.0,1,1,2022,Manchester United
46,2024-04-27,15:00,Premier League,Matchweek 35,Sat,Home,D,1,1,Burnley,...,Match Report,NaN,27.0,10.0,18.0,0.0,0,0,2022,Manchester United
47,2024-05-06,20:00,Premier League,Matchweek 36,Mon,Away,L,0,4,Crystal Palace,...,Match Report,NaN,7.0,2.0,16.7,1.0,0,0,2022,Manchester United
48,2024-05-12,16:30,Premier League,Matchweek 37,Sun,Home,L,0,1,Arsenal,...,Match Report,NaN,14.0,2.0,22.9,0.0,0,0,2022,Manchester United


In [ ]:
match_df.to_csv("matches.csv")